# Download and display cars3D

## Import

In [1]:
%load_ext autoreload
%autoreload 2

import os
PROJECT_PATH = "/projects/compures/alexandre/disdiff_adapters"
os.chdir(PROJECT_PATH)
print(os.getcwd())

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from scipy.special import digamma, gammaln
import numpy as np
from collections import Counter, defaultdict
from math import log
import seaborn as sns

from torch.utils.data import DataLoader, TensorDataset
from lightning import Trainer

from tqdm import tqdm

#DataModule
from disdiff_adapters.data_module import *
#Dataset
from disdiff_adapters.dataset import *
#Module
from disdiff_adapters.arch.multi_distillme import *
#utils
from disdiff_adapters.utils import *
#loss   
from disdiff_adapters.loss import *
#metric
from disdiff_adapters.metric import FactorVAEScore

from disdiff_adapters.arch.multi_distillme.xfactors import Xfactors
BATCH_SIZE = 2**19
LATENT_DIM_S = 126
LATENT_DIM_T = 2
is_pca = False

torch.set_float32_matmul_precision('medium')

/import/pr_compures/alexandre/disdiff_adapters


/projects/compures/alexandre/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download

In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "randall-lab/cars3d",
    cache_dir="disdiff_adapters/data/cars3d_cache",
    trust_remote_code=True
)

Generating train split: 17568 examples [00:20, 854.43 examples/s] 


In [ ]:
ds.save_to_disk("disdiff_adapters/data/cars3d")

Saving the dataset (1/1 shards): 100%|██████████| 17568/17568 [00:00<00:00, 67669.51 examples/s]


## Load dataset

In [2]:
from datasets import load_from_disk
ds = load_from_disk("disdiff_adapters/data/cars3d")
ds_tr = ds["train"]

In [3]:
images = []
labels = []
for i in tqdm(range(17568)):
    image = torch.from_numpy(np.asarray(ds_tr[i]["image"]).copy())
    label = torch.from_numpy(np.asarray(ds_tr[i]["label"]).copy()).to(torch.long)
    images.append(image)
    labels.append(label)
images_stacked = torch.stack(images)
labels_stacked = torch.stack(labels)
images_stacked = images_stacked.numpy()
labels_stacked = labels_stacked.numpy()

  0%|          | 0/17568 [00:00<?, ?it/s]

100%|██████████| 17568/17568 [00:08<00:00, 2106.59it/s]


In [4]:
perm = torch.randperm(17568).numpy()
images_stacked = images_stacked[perm]
labels_stacked = labels_stacked[perm]

In [48]:
images_tr = images_stacked[:15000]
labels_tr = labels_stacked[:15000]

images_val = images_stacked[15000:16500]
labels_val = labels_stacked[15000:16500]

images_te = images_stacked[16500:]
labels_te = labels_stacked[16500:]

In [49]:
np.savez("cars3d_train.npz", images=images_tr, labels=labels_tr)
np.savez("cars3d_val.npz", images=images_val, labels=labels_val)
np.savez("cars3d_test.npz", images=images_te, labels=labels_te)

In [52]:
np.unique(labels_stacked[:, 1])

array([0, 1, 2, 3])